In [237]:
import numpy as np
import pandas as pd
from faker import Faker

In [238]:
paper = pd.read_csv('../data/publication/papers.csv', sep=';')
author = pd.read_csv('../data/publication/authors.csv', sep=';')
paper_author = pd.read_csv('../data/publication/paper_written_by_author.csv', sep=';', dtype={'paper_id':str})

In [239]:
result_df = pd.merge(pd.merge(paper_author,author, on='author_id'),paper, on='paper_id')
result_df = result_df.rename(columns={'name': 'author'})

In [240]:
result_df = result_df.groupby('paper_id').agg({'author': lambda x: '|'.join(map(str, x)), 'title':'first', 'year':'first'}).reset_index()

In [241]:
keyword = pd.read_csv('../data/publication/keywords.csv')
paper_keyword = pd.read_csv('../data/publication/paper_has_keywords.csv', dtype={'paper_id':str})

In [242]:
result_df = pd.merge(pd.merge(paper_keyword,keyword,on='keyword_id').groupby('paper_id').agg({'keyword': lambda x: '|'.join(map(str, x))}).reset_index(),result_df, on='paper_id')

In [243]:
conference = pd.read_csv('../data/publication/conference.csv', sep=';')
conference = conference.rename(columns={'name':'venue'})
paper_conf = pd.read_csv('../data/publication/paper_published_in_conference.csv', sep=';', dtype={'paper_id':str})

In [244]:
temp1 = pd.merge(pd.merge(paper_conf, conference, on='conference_id'),result_df,on='paper_id')
temp1 = temp1.drop('conference_id', axis=1)

In [245]:
journal = pd.read_csv('../data/publication/journal.csv', sep=';')
journal = journal.rename(columns={'name':'venue'})
paper_jour = pd.read_csv('../data/publication/paper_published_in_journal.csv', sep=';', dtype={'paper_id':str})

In [246]:
temp2 = pd.merge(pd.merge(paper_jour, journal, on='journal_id'),result_df,on='paper_id')
temp2 = temp2.drop('journal_id', axis=1)

In [247]:
temp1['venue_type'] = 'Conference'
temp2['venue_type'] = 'Journal'
temp2['conf_type'] = None

In [248]:
values = ['Symposium','Regular Conference','Workshop','Expert Group']
temp1['conf_type'] = np.random.choice(values, size=len(temp1))

In [249]:
result_df = pd.concat([temp1,temp2])

In [250]:
fake = Faker()
decision = ['ACCEPTED','REJECTED']
p_type = ['Full Paper', 'Short Paper', 'Demo Paper', 'Poster']
result_df['review_text'] = [fake.text() for _ in range(len(result_df))]
result_df['head'] = [fake.unique.name() for _ in range(len(result_df))]
result_df['reviewer1'] = [fake.unique.name() for _ in range(len(result_df))]
result_df['reviewer2'] = [fake.unique.name() for _ in range(len(result_df))]
result_df['decision'] = np.random.choice(decision, size=len(result_df))
result_df['paper_type'] = np.random.choice(p_type, size=len(result_df))

In [253]:
result_df.to_csv('../data/rdf_instances.csv', index=False)